# Финальный проект по SQL

**Цель работы**: Проанализировать базу данных крупного сервиса для чтения книг по подписке

**Основые задачи**:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных**: 
База данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Таблица **books** cодержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица **authors** cодержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица **publishers** cодержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

Таблица **ratings** cодержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица **reviews** cодержит данные о пользовательских обзорах:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Исследуем таблицы — выведем первые строки

In [1]:
# Импортируем все необходимые библиотеки:
import pandas as pd 
from sqlalchemy import text, create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

In [3]:
# Выведем первые 5 строк таблицы books
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# Выведем первые 5 строк таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# Выведем первые 5 строк таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# Выведем первые 5 строк таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# Выведем первые 5 строк таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитаем, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''SELECT COUNT(book_id) AS count_book 
           FROM books 
           WHERE publication_date > '2000-01-01'
           '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count_book
0,819


После 1 января 2000 года вышло `819 книг`. 

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query = '''SELECT b.title,
                  COUNT(DISTINCT rw.review_id) AS count_reviews,
                  AVG(r.rating) AS avg_rating
           FROM books AS b 
           LEFT JOIN ratings AS r ON b.book_id = r.book_id
           LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
           GROUP BY b.book_id
           '''
pd.io.sql.read_sql(sql=text(query), con = con)

,title,count_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


Для каждой книги посчитали количество обзоров и среднюю оценку.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
query = '''SELECT p.publisher,
                  COUNT(b.book_id) as count_book
           FROM publishers AS p  
           LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE num_pages > 50 
           GROUP BY p.publisher
           ORDER BY count_book DESC
           LIMIT 1
           '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_book
0,Penguin Books,42


Издательство `Penguin Books` выпустило больше всего книг толще 50 страниц (42 книги).

## Определим автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

In [11]:
query = '''WITH rt AS (SELECT b.author_id,
                             COUNT(rating_id) AS count_score,
                             AVG(rating) AS avg_rating_book
                      FROM books AS b
                      LEFT JOIN ratings AS r ON b.book_id = r.book_id
                      GROUP BY b.book_id)
           SELECT a.author,
                  AVG(rt.avg_rating_book) AS avg_rating_books
           FROM authors AS a 
           LEFT JOIN rt ON a.author_id= rt.author_id
           WHERE count_score >= 50 
           GROUP BY a.author
           ORDER BY avg_rating_books DESC
           LIMIT 1
           '''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating_books
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг с 50 и более оценками - `J.K. Rowling/Mary GrandPré`.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''WITH rt AS (SELECT username,
                              COUNT(rating_id) AS count_score
                         FROM ratings
                         GROUP BY username),
                         
                rv AS (SELECT username,
                              COUNT(review_id) AS count_review
                         FROM reviews
                         GROUP BY username)
           
           SELECT AVG(rv.count_review) AS avg_count_review
           FROM rt 
           INNER JOIN rv ON rt.username = rv.username
           WHERE rt.count_score > 48
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count_review
0,24.0


Среднее количество обзоров от пользователей, которые поставили более 48 оценок, равно `24`.

## Общий вывод

Проанализировали базу данных крупного сервиса для чтения книг по подписке:
- Посмотрели первые 5 строк каждой из 5-ти таблиц
- Определили, что с 1 января 2000 года вышло `819 книг`
- Для каждой книги рассчитали количество обзоров и среднюю оценку
- Определили, что издательство  `Penguin Books` выпустило наибольшее число книг толще 50 страниц (42 книги)
- Выделили автора с самой высокой средней оценкой книг с 50 и более оценками - `J.K. Rowling/Mary GrandPré`
- Нашли среднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24 обзора`. 

Полученные данные могут помочь сформулировать ценностное предложение для нового продукта.
